In [2]:
import os
import random
import pathlib
from utils.hand_model import HandModel
from utils.object_model import ObjectModel
import numpy as np
import torch
from utils.hand_model_type import handmodeltype_to_joint_names, HandModelType
from utils.qpos_pose_conversion import qpos_to_pose
from utils.seed import set_seed
from plotly.subplots import make_subplots
import plotly.express as px
import plotly.graph_objects as go
from utils.joint_angle_targets import (
    OptimizationMethod,
    compute_optimized_joint_angle_targets_given_grasp_orientations,
)
from utils.parse_object_code_and_scale import parse_object_code_and_scale


In [24]:

meshdata_root_path = "../data_new_partnet/meshdata"
grasp_config_dicts_path = pathlib.Path("../data_new_partnet/")
hand_model_type = HandModelType.ALLEGRO_HAND
seed = 102
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")


In [37]:
object_code = "Dispenser_101540_merged"
grasp_data_list = np.load(os.path.join(grasp_config_dicts_path, "graspdata", f"{object_code}.npy"), allow_pickle=True)
object_model = ObjectModel(
    meshdata_root_path=meshdata_root_path,
    batch_size_each=1,
    device=device,
)

object_model.initialize(object_code, [1]) #  list(map(lambda x: x['scale'], grasp_data_list))

In [38]:
hand_model_type

<HandModelType.ALLEGRO_HAND: 'ALLEGRO_HAND'>

In [42]:
batch_idx = 10
grasp = grasp_data_list[batch_idx]
qpos = grasp['qpos']
object_scale = grasp['scale']

joint_names = handmodeltype_to_joint_names[hand_model_type]
hand_model = HandModel(hand_model_type, device=device)

hand_pose = qpos_to_pose(
    qpos=qpos, joint_names=joint_names, unsqueeze_batch_dim=True
).to(device)
hand_model.set_parameters(hand_pose)

In [44]:
hand_mesh = hand_model.get_trimesh_data(0)
object_mesh = object_model.object_mesh_list[0].copy().apply_scale(object_scale)
(hand_mesh + object_mesh).show()

```python
fig_title = f"Grasp Code: {object_code}, Index: {random_grasp_index}"
idx_to_visualize = batch_idx

fig = go.Figure(
    layout=go.Layout(
        scene=dict(
            xaxis=dict(title="X"),
            yaxis=dict(title="Y"),
            zaxis=dict(title="Z"),
            aspectmode="data",
        ),
        showlegend=True,
        title=fig_title,
        autosize=False,
        width=800,
        height=800,
    )
)
plots = [
    *hand_model.get_plotly_data(
        i=idx_to_visualize, opacity=1.0, with_contact_candidates=True
    ),
    *object_model.get_plotly_data(i=idx_to_visualize, opacity=0.5),
]
for plot in plots:
    fig.add_trace(plot)
fig.show()

# %% [markdown]
# ## Compute optimized joint angle targets

# %%
original_hand_pose = hand_model.hand_pose.detach().clone()
print(f"original_hand_pose[:, 9:] = {original_hand_pose[:, 9:]}")

# %%
assert grasp_orientations.shape == (hand_model.num_fingers, 3, 3)
(
    joint_angle_targets_to_optimize,
    debug_info,
) = compute_optimized_joint_angle_targets_given_grasp_orientations(
    joint_angles_start=original_hand_pose[:, 9:],
    hand_model=hand_model,
    grasp_orientations=grasp_orientations.unsqueeze(dim=0),
)
losses = debug_info["loss"]

# %%
fig = px.line(y=losses)
fig.update_layout(
    title=f"{joint_angle_targets_optimization_method} Loss vs. Iterations",
    xaxis_title="Iterations",
    yaxis_title="Loss",
)
fig.show()
```